In [42]:
import os
import pandas as pd
import re

Based on the output of the feature selection section, we first merge the selected features for all years. As shown in the code, the 'urban' column (to indicate rural or urban data as 0 and 1) and the 'year' column have been added to the dataframe.

In [44]:
import os
import re
import pandas as pd

def empty_string_remover(df):
    return df.replace('', pd.NA)

def add_suffix_to_columns(df, suffix):
    df.columns = [f"{col}_{suffix}" if col != "Address" else col for col in df.columns]
    return df

def read_available_columns(file, sheet_name, desired_columns):
    all_columns = pd.read_excel(file, sheet_name=sheet_name, nrows=0).columns
    available_columns = [col for col in desired_columns if col in all_columns]
    return pd.read_excel(file, sheet_name=sheet_name, usecols=available_columns)

sheet_indices = [0, 1, 2, 9, 14, 15, 17, 18, 19]

files = ['R98_cleaned.xlsx', 'U98_cleaned.xlsx', 'R99_cleaned.xlsx', 'U99_cleaned.xlsx', 'R1400_cleaned.xlsx', 'U1400_cleaned.xlsx', 'R1401_cleaned.xlsx', 'U1401_cleaned.xlsx']

sheets_desc = {
    0: "R99Data",
    1: "R99P1",
    2: "R99P2",
    9: "Transport",
    14: "Expenditure",
    15: "invesments",
    17: "R99P4S01",
    18: "R99P4S02",
    19: "R99P4S03",
}

current_directory = os.getcwd()
dfs = []

for file in files:
    file_path = os.path.join(current_directory, file)
    year_file = pd.ExcelFile(file_path, engine='openpyxl')

    Urban = "U" in file
    year = int(re.findall(r'\d+', file.split('.')[0])[0])
    if year < 100:
        year += 1300

    year_df = pd.DataFrame()

    for sheet_index in sheet_indices:
        sheet_name = year_file.sheet_names[sheet_index]

        if sheet_index == 0:
            desired_columns = ["Address", "MahMorajeh", "Fasl", "weight", "Takmil", "khanevartype", "town", "province_encoded"]
            current_df = read_available_columns(year_file, sheet_name, desired_columns)
            current_df = empty_string_remover(current_df)
            current_df["Urban"] = Urban
            current_df["Year"] = year
            year_df = current_df.copy()

        elif sheet_index == 1:
            desired_columns = ["Address", "member", "age", "relation_encoded", "gender_encoded", "studying_encoded", "degree_encoded", "occupationalst_encoded", "maritalst_encoded"]
            current_df = read_available_columns(year_file, sheet_name, desired_columns)
            current_df = empty_string_remover(current_df)

            family_member_count = current_df.groupby("Address").agg({"member": "count"}).reset_index()
            active_member_count = current_df[current_df.occupationalst_encoded == 1].groupby("Address").agg({"occupationalst_encoded": "count"}).reset_index()
            
            year_df = year_df.merge(current_df, on="Address", how="left")
            year_df = year_df.merge(family_member_count, on="Address", how="left")
            year_df = year_df.merge(active_member_count, on="Address", how="left")

        elif sheet_index == 2:
            desired_columns = ["Address", "vehicle", "room", "space", "construction", "refridgerator", "fridge", "sewingmachine", "fan", "telephone", "evapcooling", "tenure_encoded", "Material_encoded", "HeatingFuel_encoded", "waterheatingfuel_encoded"]
            current_df = read_available_columns(year_file, sheet_name, desired_columns)
            current_df = empty_string_remover(current_df)
            year_df = year_df.merge(current_df, on="Address", how="left")

        elif sheet_index == 9:
            current_df = pd.read_excel(year_file, sheet_name=sheet_name)
            current_df = empty_string_remover(current_df)
            transportation_cost_df = current_df.groupby("Address").agg({"value": "sum"}).reset_index()
            transportation_cost_df = add_suffix_to_columns(transportation_cost_df, "Transport")
            year_df = year_df.merge(transportation_cost_df, on="Address", how="left")

        elif sheet_index == 14:
            current_df = pd.read_excel(year_file, sheet_name=sheet_name)
            current_df = empty_string_remover(current_df)
            current_df['value'] = pd.to_numeric(current_df['value'], errors='coerce').fillna(0)
            expenditure_df = current_df.groupby("Address").agg({"value": "sum"}).reset_index()
            expenditure_df = add_suffix_to_columns(expenditure_df, "Expenditure")
            year_df = year_df.merge(expenditure_df, on="Address", how="left")

            insurance_expenses = current_df[current_df.code.isin([125411, 125412, 125413])].groupby("Address").agg({"value": "sum"}).reset_index()
            vehicle_expenses = current_df[current_df.code.astype("str").str.match("^7[12]\\d+$")].groupby("Address").agg({"value": "sum"}).reset_index()
            insurance_expenses = add_suffix_to_columns(insurance_expenses, "invesments")
            vehicle_expenses = add_suffix_to_columns(vehicle_expenses, "vehicle")
            year_df = year_df.merge(insurance_expenses, on="Address", how="left")
            year_df = year_df.merge(vehicle_expenses, on="Address", how="left")

        elif sheet_index == 17:
            current_df = pd.read_excel(year_file, sheet_name=sheet_name)
            current_df = empty_string_remover(current_df)
            income_azad_df = current_df.groupby("Address").agg({"income_s_y": "sum"}).reset_index()
            year_df = year_df.merge(income_azad_df, on="Address", how="left")

        elif sheet_index == 18:
            current_df = pd.read_excel(year_file, sheet_name=sheet_name)
            current_df = empty_string_remover(current_df)
            misc_income = current_df.groupby("Address").agg({
                "income_pension": "sum",
                "income_rent": "sum",
                "income_interest": "sum",
                "income_aid": "sum",
                "income_resale": "sum",
                "income_transfer": "sum"
            }).reset_index()
            year_df = year_df.merge(misc_income, on="Address", how="left")

    dfs.append(year_df)

final_df = pd.concat(dfs, ignore_index=True)
final_df.to_excel("final_merged_data.xlsx", index=False)
print(final_df.head())

       Address  MahMorajeh  Fasl    weight  khanevartype  Takmil  town  \
0  20001383908         5.0     2  5.148654             1       1     1   
1  20001383908         5.0     2  5.148654             1       1     1   
2  20001383908         5.0     2  5.148654             1       1     1   
3  20001383908         5.0     2  5.148654             1       1     1   
4  20001383911         5.0     2  5.148654             1       2     1   

   province_encoded  Urban  Year  ...  value_Expenditure  value_invesments  \
0               0.0  False  1398  ...         63213552.0               NaN   
1               0.0  False  1398  ...         63213552.0               NaN   
2               0.0  False  1398  ...         63213552.0               NaN   
3               0.0  False  1398  ...         63213552.0               NaN   
4               0.0  False  1398  ...         38560000.0               NaN   

   value_vehicle  income_s_y  income_pension  income_rent  income_interest  \
0      5

In [45]:
final_df.columns

Index(['Address', 'MahMorajeh', 'Fasl', 'weight', 'khanevartype', 'Takmil',
       'town', 'province_encoded', 'Urban', 'Year', 'member_x', 'age',
       'relation_encoded', 'gender_encoded', 'studying_encoded',
       'degree_encoded', 'occupationalst_encoded_x', 'maritalst_encoded',
       'member_y', 'occupationalst_encoded_y', 'room', 'space', 'construction',
       'vehicle', 'refridgerator', 'fridge', 'sewingmachine', 'fan',
       'telephone', 'evapcooling', 'tenure_encoded', 'Material_encoded',
       'HeatingFuel_encoded', 'waterheatingfuel_encoded', 'value_Transport',
       'value_Expenditure', 'value_invesments', 'value_vehicle', 'income_s_y',
       'income_pension', 'income_rent', 'income_interest', 'income_aid',
       'income_resale', 'income_transfer'],
      dtype='object')

در ابتدا داده تست که شامل داده های مربوط به زمستان ۱۴۰۱ است را جدا می‌کنیم

In [46]:
winter_1401_df = final_df[(final_df['Fasl'] == 4) & (final_df['Year'] == 1401)]


winter_1401_df.to_excel("Testdata.xlsx", index=False)

In [47]:
print(f"Rows in winter_1401_df (Test set): {len(winter_1401_df)}")
print(f"Rows in final_df (Before removal): {len(final_df)}")

Rows in winter_1401_df (Test set): 31207
Rows in final_df (Before removal): 516888


In [48]:
final_df = final_df[~((final_df['Fasl'] == 4) & (final_df['Year'] == 1401))]

In [49]:
print(f"Rows in final_df (After removal): {len(final_df)}")

Rows in final_df (After removal): 485681


In [53]:
# محاسبه درصد نال برای هر ستون
percent_null = (final_df.isna().sum() / len(final_df)) * 100
threshold = 5
columns_above_threshold = percent_null[percent_null > threshold]
print(columns_above_threshold)

MahMorajeh                  46.158899
relation_encoded            10.499690
occupationalst_encoded_y    13.133106
tenure_encoded              42.531826
Material_encoded            42.531826
HeatingFuel_encoded         42.532856
waterheatingfuel_encoded    42.531826
value_Transport              6.154451
value_invesments            60.671305
value_vehicle               50.858279
income_s_y                  66.562414
income_pension              13.857038
income_rent                 13.857038
income_interest             13.857038
income_aid                  13.857038
income_resale               13.857038
income_transfer             13.857038
dtype: float64


In [52]:
# پر کردن نال‌ها در province_encoded با استفاده از town
province_by_town = final_df.dropna(subset=['province_encoded']).groupby('town')['province_encoded'].apply(lambda x: x.mode()[0]).to_dict()


final_df['province_encoded'] = final_df.apply(
    lambda row: province_by_town[row['town']] if pd.isna(row['province_encoded']) else row['province_encoded'],
    axis=1
)


In [54]:
# پر کردن نال‌ها در relation_encoded با استفاده از khanevartype
relation_by_khanevartype = final_df.dropna(subset=['relation_encoded']).groupby('khanevartype')['relation_encoded'].apply(lambda x: x.mode()[0]).to_dict()

final_df['relation_encoded'] = final_df.apply(
    lambda row: relation_by_khanevartype[row['khanevartype']] if pd.isna(row['relation_encoded']) else row['relation_encoded'],
    axis=1
)

In [57]:
# پر کردن نال‌ها در occupationalst_encoded_y با استفاده از gender و age
occupation_by_gender_age = final_df.dropna(subset=['occupationalst_encoded_y']).groupby(['gender_encoded', 'age'])['occupationalst_encoded_y'].apply(lambda x: x.mode()[0]).to_dict()


final_df['occupationalst_encoded_y'] = final_df.apply(
    lambda row: occupation_by_gender_age.get((row['gender_encoded'], row['age']), row['occupationalst_encoded_y']),
    axis=1
)


In [64]:
#حذف مقادیر نال هزینه رفت و آمد(پایین بودن درصد این مقدار به ما این امکان را داد)
filtered_df = final_df.dropna(subset=['value_Transport'])

In [61]:
# محاسبه درصد نال برای هر ستون
percent_null = (filtered_df.isna().sum() / len(filtered_df)) * 100
threshold = 5
columns_above_threshold = percent_null[percent_null > threshold]
print(columns_above_threshold)


MahMorajeh                  46.014173
tenure_encoded              42.761359
Material_encoded            42.761359
HeatingFuel_encoded         42.762456
waterheatingfuel_encoded    42.761359
value_invesments            58.166919
value_vehicle               47.771342
income_s_y                  65.633954
income_pension              14.065688
income_rent                 14.065688
income_interest             14.065688
income_aid                  14.065688
income_resale               14.065688
income_transfer             14.065688
dtype: float64


به علت بالا  بودن مقادیر نال برخی از ستون ها برای جلوگیری از هرگونه سوگیری مدل آنها را حذف می‌کنیم

In [66]:
filtered_df = filtered_df.drop(columns=['income_s_y'])

In [67]:
# محاسبه درصد نال برای هر ستون
percent_null = (filtered_df.isna().sum() / len(filtered_df)) * 100
threshold = 5
columns_above_threshold = percent_null[percent_null > threshold]
print(columns_above_threshold)


MahMorajeh                  46.014173
tenure_encoded              42.761359
Material_encoded            42.761359
HeatingFuel_encoded         42.762456
waterheatingfuel_encoded    42.761359
value_invesments            58.166919
value_vehicle               47.771342
income_pension              14.065688
income_rent                 14.065688
income_interest             14.065688
income_aid                  14.065688
income_resale               14.065688
income_transfer             14.065688
dtype: float64


فایل را جهت همکاری موازی تا به اینجا ذخیره می‌کنیم

In [71]:

file_path = 'final_dataset.xlsx'

# ذخیره DataFrame به فایل اکسل
filtered_df.to_excel(file_path, index=False)

# چاپ پیام تأیید
print(f"Data saved to {file_path}")

Data saved to final_dataset.xlsx


یکی از روش های پر کردن مقادیر نال، استفاده ‌از knn imputer و mice میباشد.
به علت زمانبر بودن این مدل،با وجود ۹ ساعت زماندهی، متاسفانه به خروجی آن نتوانستیم دست پیدا کنیم.
مطمئنا استفاده از مدلهای پیچیده تر این اممکان را به ما میداد که مدل رگرسیون بهتری را آموزش دهیم

In [72]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
filtered_df[:] = imputer.fit_transform(filtered_df)


KeyboardInterrupt: 

In [ ]:
'''
from fancyimpute import IterativeImputer

imputer = IterativeImputer()
filtered_df[:] = imputer.fit_transform(filtered_df)
